# 深度学习预测模型

## 📋 概述

深度学习预测模型是股票分析中应用越来越广泛的方法，通过神经网络来学习复杂的市场模式。本章节将详细介绍深度学习预测模型在股票分析中的应用，包括LSTM、GRU、Transformer等模型。

**学习方式**：本文件是Jupyter Notebook格式，你可以边看边运行代码，通过实际代码示例理解深度学习预测模型。

**注意**：本示例使用简化的模型，实际应用中可能需要更复杂的网络结构。


In [ ]:
# 安装必要的库（如果还没有安装）
# !pip install pandas numpy matplotlib tensorflow

# 导入库
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error

# 尝试导入TensorFlow
try:
    import tensorflow as tf
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import LSTM, GRU, Dense, Dropout, Attention
    from tensorflow.keras.optimizers import Adam
    TENSORFLOW_AVAILABLE = True
    print("TensorFlow可用，将使用深度学习模型")
except ImportError:
    TENSORFLOW_AVAILABLE = False
    print("TensorFlow不可用，将使用简化的模拟模型")

# 设置中文字体
plt.rcParams['font.sans-serif'] = ['SimHei', 'Arial Unicode MS', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False

print("环境准备完成！")


## 📖 核心内容

### 1. 深度学习预测模型概述

#### 1.1 常用模型

**循环神经网络**：
- LSTM（长短期记忆网络）
- GRU（门控循环单元）

**注意力机制**：
- Transformer
- Attention机制

**卷积神经网络**：
- 1D CNN用于时间序列

让我们创建一个LSTM模型来预测股票价格：


In [ ]:
# 生成示例数据
np.random.seed(42)
n_days = 1000
dates = pd.date_range(start='2021-01-01', periods=n_days, freq='D')
dates = dates[dates.weekday < 5]

trend = np.linspace(10, 15, len(dates))
noise = np.cumsum(np.random.randn(len(dates)) * 0.1)
prices = trend + noise

price_data = pd.Series(prices, index=dates, name='Close')

# 准备LSTM数据
def create_sequences(data, seq_length=60):
    """创建时间序列数据"""
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length])
    return np.array(X), np.array(y)

# 标准化
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_prices = scaler.fit_transform(price_data.values.reshape(-1, 1))

# 创建序列
seq_length = 60
X, y = create_sequences(scaled_prices.flatten(), seq_length)

# 划分训练集和测试集
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# 调整维度
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

print(f"数据准备完成！")
print(f"训练集：{len(X_train)} 个样本")
print(f"测试集：{len(X_test)} 个样本")

if TENSORFLOW_AVAILABLE:
    # 构建LSTM模型
    lstm_model = Sequential([
        LSTM(50, return_sequences=True, input_shape=(seq_length, 1)),
        Dropout(0.2),
        LSTM(50, return_sequences=False),
        Dropout(0.2),
        Dense(1)
    ])
    
    lstm_model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
    
    print("\nLSTM模型结构：")
    lstm_model.summary()
    
    # 训练模型
    print("\n开始训练LSTM模型...")
    history = lstm_model.fit(
        X_train, y_train,
        batch_size=32,
        epochs=10,  # 实际应用需要更多epochs
        validation_split=0.2,
        verbose=1
    )
    
    # 预测
    train_pred = lstm_model.predict(X_train)
    test_pred = lstm_model.predict(X_test)
    
    # 反标准化
    train_pred = scaler.inverse_transform(train_pred)
    y_train_actual = scaler.inverse_transform(y_train.reshape(-1, 1))
    test_pred = scaler.inverse_transform(test_pred)
    y_test_actual = scaler.inverse_transform(y_test.reshape(-1, 1))
    
    # 评估
    train_mse = mean_squared_error(y_train_actual, train_pred)
    test_mse = mean_squared_error(y_test_actual, test_pred)
    test_mae = mean_absolute_error(y_test_actual, test_pred)
    
    print(f"\n模型评估：")
    print(f"训练集MSE: {train_mse:.4f}")
    print(f"测试集MSE: {test_mse:.4f}")
    print(f"测试集MAE: {test_mae:.4f}")
    
    # 可视化
    fig, axes = plt.subplots(2, 1, figsize=(15, 10))
    
    # 训练过程
    axes[0].plot(history.history['loss'], label='训练损失', linewidth=2)
    axes[0].plot(history.history['val_loss'], label='验证损失', linewidth=2)
    axes[0].set_title('LSTM模型训练过程', fontsize=14, fontweight='bold')
    axes[0].set_xlabel('Epoch')
    axes[0].set_ylabel('Loss')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # 预测结果
    test_dates = price_data.index[train_size + seq_length:train_size + seq_length + len(y_test)]
    axes[1].plot(test_dates, y_test_actual.flatten(), label='实际价格', 
                linewidth=2, color='black', alpha=0.7)
    axes[1].plot(test_dates, test_pred.flatten(), label='LSTM预测', 
                linewidth=2, color='green', alpha=0.7)
    axes[1].set_title('LSTM价格预测结果', fontsize=14, fontweight='bold')
    axes[1].set_xlabel('日期')
    axes[1].set_ylabel('价格')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
    
else:
    print("\n注意：TensorFlow不可用")
    print("安装TensorFlow后可以运行深度学习模型：pip install tensorflow")
